In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
builder = tfds.builder("scientific_papers", data_dir="./")

In [3]:
builder.download_and_prepare()

In [4]:
ds = builder.as_dataset(split="train");

In [5]:
ds

<PrefetchDataset shapes: {abstract: (), article: (), section_names: ()}, types: {abstract: tf.string, article: tf.string, section_names: tf.string}>

In [6]:
#ds = ds.take(1)

In [7]:
print(ds)
num_elements = tf.data.experimental.cardinality(ds).numpy()

<PrefetchDataset shapes: {abstract: (), article: (), section_names: ()}, types: {abstract: tf.string, article: tf.string, section_names: tf.string}>


In [8]:
num_elements = tf.data.experimental.cardinality(ds).numpy()

In [9]:
num_elements # 203037 elements au total dans le dataset

203037

In [10]:
for element in ds:
    num_elements += 1

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/noobzik/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 737, in _next_internal
    return self._element_spec._from_compatible_tensor_list(ret)  # pylint: disable=protected-access
AttributeError: 'dict' object has no attribute '_from_compatible_tensor_list'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/noobzik/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-4fa73d30c537>", line 1, in <module>
    for element in ds:
  File "/home/noobzik/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 747, in __next__
    return self._next_internal()
  File "/home/noobzik/.local/lib/python3.8/site-packages/tensorflow/python/data/ops/iterator_ops.py", line 739, in _next_internal
    return 


KeyboardInterrupt



In [ ]:
for example in ds:  # example is {'image': tf.Tensor, 'label': tf.Tensor}
    abstract = example["abstract"]
    article = example["article"]
    section_names = example["section_names"]
    print(abstract)

In [21]:
def generate_csv(n:int=203037):
    builder = tfds.builder("scientific_papers", data_dir="./")
    builder.download_and_prepare()
    val_size = int(n/20)
    output_file = 'out_train.csv'
    output_file_val = 'out_val.csv'
    ds = builder.as_dataset(split="train")
    ds = ds.take(n-val_size)
    print(ds)
    with open(output_file, 'w') as f:
        f.write("text,summary\n")
        for example in ds:  # example is {'image': tf.Tensor, 'label': tf.Tensor}
            abstract = example["abstract"]
            article = example["article"]
            f.write('"'+bytes.decode(article.numpy()).replace('"', '\'').replace('\n', ' ') + '" ,'+'"' + bytes.decode(abstract.numpy()).replace('\n', ' ').replace('"', '\'')+'"'+ "\n")
    ds = builder.as_dataset(split="validation")
    ds = ds.take(val_size)
    with open(output_file_val, 'w') as f:
        f.write("text,summary\n")
        for example in ds:  # example is {'image': tf.Tensor, 'label': tf.Tensor}
            abstract = example["abstract"]
            article = example["article"]
            f.write('"'+bytes.decode(article.numpy()).replace('"', '\'').replace('\n', ' ') + '" ,'+'"' + bytes.decode(abstract.numpy()).replace('\n', ' ').replace('"', '\'')+'"'+ "\n")

In [22]:
generate_csv()

<TakeDataset shapes: {abstract: (), article: (), section_names: ()}, types: {abstract: tf.string, article: tf.string, section_names: tf.string}>


In [ ]:
!python3 run_summarisation.py \
    --model_name_or_path t5-small \
    --do_train \
    --do_eval \
    --train_file out_train_pd.csv \
    --validation_file out_val_pd.csv \
    --output_dir ../tst-summarization \
    --per_device_train_batch_size=4 \
    --per_device_eval_batch_size=4 \
    --predict_with_generate \
    --tf32 \
    --num_train_epochs=50 \
    --eval_steps=10000 \
    --save_total_limit=2 \
    --evaluation_strategy epoch


2021-12-14 17:44:45.343159: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
12/14/2021 17:44:49 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
12/14/2021 17:44:49 - INFO - __main__ - Training/evaluation parameters Seq2SeqTrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
bf16=False,
bf16_full_eval=False,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_steps=10000,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
generation_max_length=None,
generation_num_beams=None,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_bette